In [ ]:
!pip install torch --index-url https://download.pytorch.org/whl/cu121
!pip install transformers
!pip install datasets

Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
import json
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# Load dataset from uploaded file
dataset_path = "gradingnew.json"

with open(dataset_path, "r") as file:
    dataset = json.load(file)

# Convert dataset into text-to-text format
data = []
for item in dataset:
    question = item["question"]
    for ans in item["answers"]:
        input_text = f"Grade the following answer:\nQuestion: {question}\nStudent Answer: {ans['text']}"
        target_text = f"{ans['score']} | {ans['feedback']}"
        data.append({"input_text": input_text, "target_text": target_text})

# Split into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_dict({
    "input_text": [x["input_text"] for x in train_data],
    "target_text": [x["target_text"] for x in train_data],
})

val_dataset = Dataset.from_dict({
    "input_text": [x["input_text"] for x in val_data],
    "target_text": [x["target_text"] for x in val_data],
})

dataset_dict = DatasetDict({"train": train_dataset, "validation": val_dataset})

# Load T5 Tokenizer and Model
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Tokenization function
def preprocess_data(examples):
    inputs = tokenizer(examples["input_text"], padding="max_length", truncation=True, max_length=512)
    targets = tokenizer(examples["target_text"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply tokenization
tokenized_datasets = dataset_dict.map(preprocess_data, batched=True)

# Debug: Check if tokenized dataset has proper inputs
print("\n🔹 Sample Tokenized Input:", tokenizer.decode(tokenized_datasets["train"][0]["input_ids"]))

# Set Training Arguments (optimized for stability)
training_args = TrainingArguments(
    output_dir="./grading_model",
    evaluation_strategy="epoch",  # Enable evaluation
    save_strategy="epoch",
    learning_rate=3e-4,  # Reduce learning rate for better training stability
    per_device_train_batch_size=4,  # Reduce batch size for less memory usage
    per_device_eval_batch_size=4,
    num_train_epochs=5,  # Increase epochs for better learning
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
)

# Train the Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

trainer.train()

# Save Fine-Tuned Model
trainer.save_model("./model")
tokenizer.save_pretrained("./model")

print("✅ Training completed! Model saved at ./model")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).



🔹 Sample Tokenized Input: Grade the following answer: Question: What is the difference between a compiled language and an interpreted language? Student Answer: Compiled languages convert source code into machine code before execution, while interpreted languages execute code directly.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

<ipython-input-4-36388608ce0c>:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.201900,0.211539
2,0.192700,0.189859
3,0.136700,0.186499
4,0.188900,0.181777
5,0.125800,0.180989


✅ Training completed! Model saved at ./model


In [ ]:
model_path = "./model"  # Update path if needed
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

def evaluate_answer(question, student_answer):
    """Passes input through the model and decodes output"""
    input_text = f"Grade the following answer:\nQuestion: {question}\nStudent Answer: {student_answer}"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

    # Generate output
    outputs = model.generate(**inputs, max_length=50)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return result

# Define test cases
test_cases = [
    ("What is the difference between a list and a tuple in Python?",
     "A list is mutable, while a tuple is immutable. Lists are slower than tuples."),

    ("What is the difference between a list and a tuple in Python?",
     "Lists are changeable, but tuples are not. Lists use square brackets, tuples use parentheses."),

    ("What is the difference between a list and a tuple in Python?",
     "Lists and tuples are the same and can be modified."),

    ("What is the difference between a list and a tuple in Python?",
     "Lists and tuples store data. Lists are mutable, meaning they can be changed, while tuples are immutable. Lists use square brackets, whereas tuples use parentheses. Tuples are slightly faster."),

    ("What is the difference between a list and a tuple in Python?",
     "Lists can change, tuples can't.")
]

# Run tests
print("🔹 Model Test Results:")
for i, (question, answer) in enumerate(test_cases):
    result = evaluate_answer(question, answer)
    print(f"\nTest {i+1}:")
    print(f"📝 Student Answer: {answer}")
    print(f"🔹 Model Output: {result}")


🔹 Model Test Results:

Test 1:
📝 Student Answer: A list is mutable, while a tuple is immutable. Lists are slower than tuples.
🔹 Model Output: 5 | Excellent! Clearly explains both mutability and speed.

Test 2:
📝 Student Answer: Lists are changeable, but tuples are not. Lists use square brackets, tuples use parentheses.
🔹 Model Output: 5 | Excellent! Clearly explains the difference.

Test 3:
📝 Student Answer: Lists and tuples are the same and can be modified.
🔹 Model Output: 4 | Good answer! Some details could be expanded upon.

Test 4:
📝 Student Answer: Lists and tuples store data. Lists are mutable, meaning they can be changed, while tuples are immutable. Lists use square brackets, whereas tuples use parentheses. Tuples are slightly faster.
🔹 Model Output: 5 | Excellent! Clearly explains both mutability and tuples.

Test 5:
📝 Student Answer: Lists can change, tuples can't.
🔹 Model Output: 5 | Excellent! Clearly explains the difference.


In [ ]:
test_cases = [
    ("What are instance and class variables in Python?",
     "Instance variables are stored in a function, class variables are not."),

    ("What is the difference between a list and a tuple in Python?",
     "Lists are changeable, but tuples are not. Lists use square brackets, tuples use parentheses."),

    ("What is k-means clustering?",
     "It is an algorithm"),

    ("What is k-means clustering?",
     "It is a clustering algorithm that forms clusters based on similarity in data points."),

    ("What is k-means clustering?",
     "K-means clustering is a supervised learning algorithm used for classification problems.")
]

# Run tests
print("🔹 Model Test Results:")
for i, (question, answer) in enumerate(test_cases):
    result = evaluate_answer(question, answer)
    print(f"\nTest {i+1}:")
    print(f"📝 Student Answer: {answer}")
    print(f"🔹 Model Output: {result}")


🔹 Model Test Results:

Test 1:
📝 Student Answer: Instance variables are stored in a function, class variables are not.
🔹 Model Output: 5 | Excellent! Clearly defines instance variables.

Test 2:
📝 Student Answer: Lists are changeable, but tuples are not. Lists use square brackets, tuples use parentheses.
🔹 Model Output: 5 | Excellent! Clearly explains the difference.

Test 3:
📝 Student Answer: It is an algorithm
🔹 Model Output: 0 | Incorrect. It is a model of clustering.

Test 4:
📝 Student Answer: It is a clustering algorithm that forms clusters based on similarity in data points.
🔹 Model Output: 5 | Excellent! Well-explained with clarity.

Test 5:
📝 Student Answer: K-means clustering is a supervised learning algorithm used for classification problems.
🔹 Model Output: 5 | Excellent! Well-explained with clarity.


In [ ]:
import json
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# Load dataset from uploaded file
dataset_path = "gradingnew.json"

with open(dataset_path, "r") as file:
    dataset = json.load(file)

# Convert dataset into text-to-text format
data = []
for item in dataset:
    question = item["question"]
    for ans in item["answers"]:
        input_text = f"Grade the following answer like a data science and python teacher:\nQuestion: {question}\nStudent Answer: {ans['text']}"
        target_text = f"{ans['score']} | {ans['feedback']}"
        data.append({"input_text": input_text, "target_text": target_text})


# Split into train, validation, and test sets
train_data, temp_data = train_test_split(data, test_size=0.2, random_state=42)  # 70% train
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)  # 15% val, 15% test


# Convert to Hugging Face Dataset format (including test set)
train_dataset = Dataset.from_dict({
    "input_text": [x["input_text"] for x in train_data],
    "target_text": [x["target_text"] for x in train_data],
})

val_dataset = Dataset.from_dict({
    "input_text": [x["input_text"] for x in val_data],
    "target_text": [x["target_text"] for x in val_data],
})

test_dataset = Dataset.from_dict({
    "input_text": [x["input_text"] for x in test_data],
    "target_text": [x["target_text"] for x in test_data],
})

dataset_dict = DatasetDict({"train": train_dataset, "validation": val_dataset, "test": test_dataset})


# Load T5 Tokenizer and Model
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Tokenization function
def preprocess_data(examples):
    inputs = tokenizer(examples["input_text"], padding="max_length", truncation=True, max_length=512)
    targets = tokenizer(examples["target_text"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply tokenization
tokenized_datasets = dataset_dict.map(preprocess_data, batched=True)

# Debug: Check if tokenized dataset has proper inputs
print("\n🔹 Sample Tokenized Input:", tokenizer.decode(tokenized_datasets["train"][0]["input_ids"]))

# Set Training Arguments (optimized for stability)
training_args = TrainingArguments(
    output_dir="./grading_model1",
    evaluation_strategy="epoch",  # Enable evaluation
    save_strategy="epoch",
    learning_rate=3e-4,  # Reduce learning rate for better training stability
    per_device_train_batch_size=4,  # Reduce batch size for less memory usage
    per_device_eval_batch_size=4,
    num_train_epochs=5,  # Increase epochs for better learning
    weight_decay=0.01,
    logging_dir="./logs1",
    logging_steps=10,
    save_total_limit=2,
)

# Train the Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

trainer.train()

# Save Fine-Tuned Model
#trainer.save_model("./model")
#tokenizer.save_pretrained("./model")

#print("✅ Training completed! Model saved at ./model")

Map:   0%|          | 0/742 [00:00<?, ? examples/s]

Map:   0%|          | 0/93 [00:00<?, ? examples/s]

Map:   0%|          | 0/93 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).



🔹 Sample Tokenized Input: Grade the following answer: Question: What is the difference between a compiled language and an interpreted language? Student Answer: Compiled languages convert source code into machine code before execution, while interpreted languages execute code directly.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

<ipython-input-7-674e4b77114f>:81: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.201900,0.213620
2,0.192700,0.190988
3,0.136700,0.187734
4,0.188900,0.182611
5,0.125800,0.180845


TrainOutput(global_step=930, training_loss=0.25436644284955917, metrics={'train_runtime': 213.7912, 'train_samples_per_second': 17.353, 'train_steps_per_second': 4.35, 'total_flos': 502118083461120.0, 'train_loss': 0.25436644284955917, 'epoch': 5.0})

In [ ]:
# Evaluate on test dataset
test_results = trainer.evaluate(tokenized_datasets["test"])
print(f"🔹 Test Set Evaluation: {test_results}")


🔹 Test Set Evaluation: {'eval_loss': 0.18113331496715546, 'eval_runtime': 1.4397, 'eval_samples_per_second': 64.599, 'eval_steps_per_second': 16.671, 'epoch': 5.0}


In [ ]:
model_path = "./model"  # Update path if needed
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

def evaluate_answer(question, student_answer):
    """Passes input through the model and decodes output"""
    input_text = f"Grade the following answer like a teacher:\nQuestion: {question}\nStudent Answer: {student_answer}"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

    # Generate output
    outputs = model.generate(**inputs, max_length=50)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return result

# Define test cases
test_cases = [
    ("What is the difference between a list and a tuple in Python?",
     "A list is mutable, while a tuple is immutable. Lists are slower than tuples."),

    ("What is the difference between a list and a tuple in Python?",
     "Lists are changeable, but tuples are not. Lists use square brackets, tuples use parentheses."),

    ("What is the difference between a list and a tuple in Python?",
     "Lists and tuples are the same and can be modified."),

    ("What is the difference between a list and a tuple in Python?",
     "Lists and tuples store data. Lists are mutable, meaning they can be changed, while tuples are immutable. Lists use square brackets, whereas tuples use parentheses. Tuples are slightly faster."),

    ("What is the difference between a list and a tuple in Python?",
     "Lists can change, tuples can't.")
]

# Run tests
print("🔹 Model Test Results:")
for i, (question, answer) in enumerate(test_cases):
    result = evaluate_answer(question, answer)
    print(f"\nTest {i+1}:")
    print(f"📝 Student Answer: {answer}")
    print(f"🔹 Model Output: {result}")


🔹 Model Test Results:

Test 1:
📝 Student Answer: A list is mutable, while a tuple is immutable. Lists are slower than tuples.
🔹 Model Output: 3 | Partially correct, but missing some key aspects.

Test 2:
📝 Student Answer: Lists are changeable, but tuples are not. Lists use square brackets, tuples use parentheses.
🔹 Model Output: 3 | Partially correct, but missing some key aspects.

Test 3:
📝 Student Answer: Lists and tuples are the same and can be modified.
🔹 Model Output: 3 | Partially correct, but missing some key aspects.

Test 4:
📝 Student Answer: Lists and tuples store data. Lists are mutable, meaning they can be changed, while tuples are immutable. Lists use square brackets, whereas tuples use parentheses. Tuples are slightly faster.
🔹 Model Output: 5 | Excellent! Well-explained with clarity.

Test 5:
📝 Student Answer: Lists can change, tuples can't.
🔹 Model Output: 3 | Partially correct, but missing some key aspects.


In [ ]:
test_cases = [
    ("What are instance and class variables in Python?",
     "Instance variables are stored in a function, class variables are not."),

    ("What is the difference between a list and a tuple in Python?",
     "Lists are changeable, but tuples are not. Lists use square brackets, tuples use parentheses."),

    ("What is k-means clustering?",
     "It is an algorithm"),

    ("What is k-means clustering?",
     "It is a clustering algorithm that forms clusters based on similarity in data points."),

    ("What is k-means clustering?",
     "K-means clustering is a supervised learning algorithm used for classification problems.")
]

# Run tests
print("🔹 Model Test Results:")
for i, (question, answer) in enumerate(test_cases):
    result = evaluate_answer(question, answer)
    print(f"\nTest {i+1}:")
    print(f"📝 Student Answer: {answer}")
    print(f"🔹 Model Output: {result}")


🔹 Model Test Results:

Test 1:
📝 Student Answer: Instance variables are stored in a function, class variables are not.
🔹 Model Output: 3 | Partially correct, but missing some key aspects.

Test 2:
📝 Student Answer: Lists are changeable, but tuples are not. Lists use square brackets, tuples use parentheses.
🔹 Model Output: 3 | Partially correct, but missing some key aspects.

Test 3:
📝 Student Answer: It is an algorithm
🔹 Model Output: 0 | Incorrect. Please review the concept carefully.

Test 4:
📝 Student Answer: It is a clustering algorithm that forms clusters based on similarity in data points.
🔹 Model Output: 5 | Excellent! Well-explained with clarity.

Test 5:
📝 Student Answer: K-means clustering is a supervised learning algorithm used for classification problems.
🔹 Model Output: 4 | Good answer! Some details could be expanded upon.


In [ ]:
# Save Fine-Tuned Model
trainer.save_model("./model1")
tokenizer.save_pretrained("./model1")

#print("✅ Training completed! Model saved at ./model")

('./model1/tokenizer_config.json',
 './model1/special_tokens_map.json',
 './model1/spiece.model',
 './model1/added_tokens.json')

In [ ]:
model_path = "./model1"  # Update path if needed
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

def evaluate_answer(question, student_answer):
    """Passes input through the model and decodes output"""
    input_text = f"Grade the following answer like a teacher grading an exam:\nQuestion: {question}\nStudent Answer: {student_answer}"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

    # Generate output
    outputs = model.generate(**inputs, max_length=50)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return result

# Define test cases
test_cases = [
    ("What is the difference between a list and a tuple in Python?",
     "A list is mutable, while a tuple is immutable. Lists are slower than tuples."),

    ("What is the difference between a list and a tuple in Python?",
     "Lists are changeable, but tuples are not. Lists use square brackets, tuples use parentheses."),

    ("What is the difference between a list and a tuple in Python?",
     "Lists and tuples are the same and can be modified."),

    ("What is the difference between a list and a tuple in Python?",
     "Lists and tuples store data. Lists are mutable, meaning they can be changed, while tuples are immutable. Lists use square brackets, whereas tuples use parentheses. Tuples are slightly faster."),

    ("What is the difference between a list and a tuple in Python?",
     "Lists can change, tuples can't.")
]

# Run tests
print("🔹 Model Test Results:")
for i, (question, answer) in enumerate(test_cases):
    result = evaluate_answer(question, answer)
    print(f"\nTest {i+1}:")
    print(f"📝 Student Answer: {answer}")
    print(f"🔹 Model Output: {result}")

🔹 Model Test Results:

Test 1:
📝 Student Answer: A list is mutable, while a tuple is immutable. Lists are slower than tuples.
🔹 Model Output: 4 | Good answer! Some details could be expanded upon.

Test 2:
📝 Student Answer: Lists are changeable, but tuples are not. Lists use square brackets, tuples use parentheses.
🔹 Model Output: 4 | Good answer! Some details could be expanded upon.

Test 3:
📝 Student Answer: Lists and tuples are the same and can be modified.
🔹 Model Output: 3 | Partially correct, but missing some key aspects.

Test 4:
📝 Student Answer: Lists and tuples store data. Lists are mutable, meaning they can be changed, while tuples are immutable. Lists use square brackets, whereas tuples use parentheses. Tuples are slightly faster.
🔹 Model Output: 5 | Excellent! Clearly explains both mutability and tuples.

Test 5:
📝 Student Answer: Lists can change, tuples can't.
🔹 Model Output: 4 | Good answer! Some details could be expanded upon.


In [ ]:
test_cases = [
    ("What are instance and class variables in Python?",
     "Instance variables are stored in a function, class variables are not."),

    ("What is the difference between a list and a tuple in Python?",
     "Lists are changeable, but tuples are not. Lists use square brackets, tuples use parentheses."),

    ("What is k-means clustering?",
     "It is an algorithm"),

    ("What is k-means clustering?",
     "It is a clustering algorithm that forms clusters based on similarity in data points."),

    ("What is k-means clustering?",
     "K-means clustering is a supervised learning algorithm used for classification problems.")
]

# Run tests
print("🔹 Model Test Results:")
for i, (question, answer) in enumerate(test_cases):
    result = evaluate_answer(question, answer)
    print(f"\nTest {i+1}:")
    print(f"📝 Student Answer: {answer}")
    print(f"🔹 Model Output: {result}")


🔹 Model Test Results:

Test 1:
📝 Student Answer: Instance variables are stored in a function, class variables are not.
🔹 Model Output: 4 | Good answer! Some details could be expanded upon.

Test 2:
📝 Student Answer: Lists are changeable, but tuples are not. Lists use square brackets, tuples use parentheses.
🔹 Model Output: 4 | Good answer! Some details could be expanded upon.

Test 3:
📝 Student Answer: It is an algorithm
🔹 Model Output: 2 | Needs improvement. Consider covering the concept more clearly.

Test 4:
📝 Student Answer: It is a clustering algorithm that forms clusters based on similarity in data points.
🔹 Model Output: 5 | Excellent! Well-explained with clarity.

Test 5:
📝 Student Answer: K-means clustering is a supervised learning algorithm used for classification problems.
🔹 Model Output: 5 | Excellent! Well-explained with clarity.


In [ ]:
import shutil

# Define model save path
model_dir = "t5_finetuned_grading"

# Save model and tokenizer
trainer.save_model(model_dir)
tokenizer.save_pretrained(model_dir)

# Zip the directory for easy download
shutil.make_archive(model_dir, 'zip', model_dir)

print(f"✅ Model saved and zipped as {model_dir}.zip")

✅ Model saved and zipped as t5_finetuned_grading.zip
